# import the neccessary module and load data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sb
from datetime import date
import matplotlib.pyplot as plt
from sklearn.metrics import plot_confusion_matrix
import tkinter
from tkinter import *
from PIL import ImageTk, Image
import string

data=pd.read_excel('Data.xlsx')

# data['Tiket Status '].value_counts()


In [ ]:
data

# Generate Age

In [ ]:
def calculateAge(birthdate):
    today = date.today()
    age = today.year - birthdate.year - ((today.month, today.day) < (birthdate.month, birthdate.day))
    return age

for index, row in data.iterrows():
    age=calculateAge(row['dateOfBirth'])
    if(age<14):
        data.iloc[index,4]="Children"
    elif(age<25):
        data.iloc[index,4]="Youth"
    elif(age<65):
        data.iloc[index,4]="Adult"
    else:
        data.iloc[index,4]="Senior"

# Generate Quarter Number

In [ ]:
df=data
for index, row in df.iterrows():
    RDate=row['Tiketed_Date']
    month=pd.to_datetime(RDate).month
    if(month<=3):
        month=1
    elif(month<=6):
        month=2
    elif(month<=9):
        month=3
    else:
        month=4
    df.loc[df['Tiketed_Date'] == RDate,'Tiketed_Date']=month

In [ ]:
data

# Drop irrelevant column and clean data 

In [ ]:
cols_to_drop=['miles','Rescheduled_Date','Date_of_Cancelation','title','Ticket First time Use','Rescheduled_Date','Refund Status','dateOfBirth']
df=data.drop(cols_to_drop,axis=1)

In [ ]:
df

# Modify Class of service

In [ ]:
df.loc[df['cabin_class of service'] == 'Economy Class','cabin_class of service']='Economy'
df.loc[df['cabin_class of service'] == 'Business Class','cabin_class of service']='Business'
df.loc[df['Nationality'] == 'Ethiopia','Nationality']='Local'
df.loc[df['Nationality'] != 'Local','Nationality']='Foreign'

In [ ]:
df

# Check for null value

In [ ]:
sb.heatmap(df.isnull())

# Filling Missing value

In [ ]:
df["gender"].fillna(df['gender'].mode()[0], inplace = True)
df["Nationality"].fillna(df['Nationality'].mode()[0], inplace = True)
df["Age"].fillna(df['Age'].mode()[0], inplace = True)
df["Fligth_ID"].fillna(df['Fligth_ID'].mode()[0], inplace = True)
df["cabin_class of service"].fillna(df['cabin_class of service'].mode()[0], inplace = True)
df["Reservation Status"].fillna(df['Reservation Status'].mode()[0], inplace = True)
df["Date_of_Reservation"].fillna(df['Date_of_Reservation'].mode()[0], inplace = True)
df["Tiketed_Date"].fillna(df['Tiketed_Date'].mode()[0], inplace = True)

In [ ]:
df['Fligth_ID'].mode()[0]

In [ ]:
sb.heatmap(df.isnull())

# Data Cleaning 

In [ ]:
df.loc[df['Tiket Status '] == 'Ticketed Used','Tiket Status ']='Tickete Used'
df.loc[df['Tiket Status '] == 'Canceled','Tiket Status ']='Tickete Not Used'
df.loc[df['Tiket Status '] == 'Not Ticketed','Tiket Status ']='Tickete Not Used'
df.loc[df['Nationality'] == 'Ethiopia','Nationality']='Local'
df.loc[df['Nationality'] != 'Local','Nationality']='Foreign'
# 0 Canceled
# 1 Exchanged
# 2 Not Ticketed
# 3 Refunded
# 4 Ticketed Used
# 5 Tiketed  Not Used
df

# Convert Text categories in to numeric categories

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert categorical value to number 
le_Age=LabelEncoder()
le_gender=LabelEncoder()
le_nationality=LabelEncoder()
le_flightID=LabelEncoder()
le_cabinClass=LabelEncoder()
le_reservationStatus=LabelEncoder()
le_ticket_status=LabelEncoder()

df['N_Age']=le_Age.fit_transform(df['Age'])
df['N_Gender']=le_nationality.fit_transform(df['gender'])
df['N_Nationality']=le_nationality.fit_transform(df['Nationality'])
df['N_FlightID']=le_nationality.fit_transform(df['Fligth_ID'])
df['N_CabinClass']=le_nationality.fit_transform(df['cabin_class of service'])
df['N_ReservationStatus']=le_nationality.fit_transform(df['Reservation Status'])
df['N_TicketStatus']=le_nationality.fit_transform(df['Tiket Status '])

# convert date or timestamp to number 

le_ReservationDate=LabelEncoder()

df['N_ReservationDate']=le_nationality.fit_transform(df['Date_of_Reservation'])

before_drop=df;


# Categorize Date into month interval

In [ ]:
for index, row in df.iterrows():
    RDate=row['N_ReservationDate']
    if(RDate<30):
        df.iloc[index,16]=0
    elif(RDate<60):
        df.iloc[index,16]=1
    elif(RDate<90):
        df.iloc[index,16]=2
    elif(RDate>=90):
        df.iloc[index,16]=3


In [ ]:
df

In [ ]:
pd.options.display.min_rows=300
df

# Drop categorical value

In [ ]:
cols_to_drop=['Age','gender','Nationality','Fligth_ID','cabin_class of service','Reservation Status','Tiket Status ','Date_of_Reservation']
new_df=df.drop(cols_to_drop,axis=1)


In [ ]:
new_df.head(25)

# Split the Dataset in to training and test data set

In [ ]:
from sklearn.model_selection import train_test_split

input_x=new_df[['N_Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','Tiketed_Date']]
target_y=new_df['N_TicketStatus']

x_train,x_test,y_train,y_test=train_test_split(input_x,target_y,test_size=0.3,random_state=0)


In [ ]:
x_train

In [ ]:
x_test

# Build Decission Tree Classifier model 

In [ ]:
from sklearn import tree

# Build
model=tree.DecisionTreeClassifier() 




# Train Data 

In [ ]:
testY=model.fit(input_x,target_y)
actual=model.score(x_train,y_train)
actual

# Test Data

In [ ]:
predict=model.predict(x_test)
model.score(x_test,y_test)


# Accuracy Test

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predict)

# Confusion matrix 

In [ ]:
from sklearn.metrics import confusion_matrix

matrix=plot_confusion_matrix(model,x_test,y_test,cmap=plt.cm.Reds)
# matrix.ax_.set_title('Confusion Matrix',color='white')
# plt.xlabel('True Label',color='white')
# plt.ylabel('True Label',color='white')
plt.gcf().axes[0].tick_params(color='white')
plt.gcf().axes[1].tick_params(color='white')
plt.gcf().set_size_inches(10,5)
plt.show()
# confusion_matrix(y_test, predict)
# confusionchart(confusion_matrix(y_true, y_pred))
# correct=27+2+27+207+6
# incorrect=2+1
# total=274 #the total is y_test length
# accuray=correct/total


In [ ]:
import numpy as np
from sklearn.metrics import classification_report

print(classification_report(y_test, predict))
# 0 Canceled
# 1 Exchanged
# 2 Not Ticketed
# 3 Refunded
# 4 Ticketed Used
# 5 Tiketed  Not Used

# Random prediction for test

In [ ]:
# to do function
# 'N_Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','Tiketed_Date'

print(model.predict([[0,0,1,0,1,4]]))

In [ ]:
#new_df

In [ ]:
df['N_TicketStatus'].value_counts()
# 0 Exchanged
# 1 Refunded              
# 2 Tiketed  Not Used             
# 3 Tickete Used


In [ ]:
before_drop['Tiket Status '].value_counts()

In [ ]:
test=before_drop.sort_values(['Nationality'])
test.head(50)


In [ ]:
before_drop.loc[before_drop['Nationality'] == 'Ethiopia','Nationality']='Local'
before_drop.loc[before_drop['Nationality'] != 'Local','Nationality']='Foreign'

In [ ]:
before_drop

In [ ]:
before_drop.groupby(['Tiket Status ']).count()

# Gender with Ticket Status Aggregation

In [ ]:
genderTicketAggregated= before_drop.groupby(['Tiket Status ','gender'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','Nationality','Fligth_ID','cabin_class of service','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=genderTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
genderDf = pd.DataFrame(columns=["Index","Gender","Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
genderDf[genderDf.columns[0]] = ["Female", "Male"]
test=[]
for index, row in result.iterrows():
    test.append(row[0])
j=0;
for i in range(len(test)):
    this_column = genderDf.columns[i+1]
    genderDf[this_column] = [test[j], test[j+1]]
    j=j+2
    if(i==3):
        break
genderDf.to_excel("GenderTicketStatusAggregationResult.xlsx",index=False)
genderTicket=pd.read_excel('GenderTicketStatusAggregationResult.xlsx')
genderTicketTranspose=genderTicket.T
genderTicketTranspose=genderTicketTranspose.iloc[1: , :]
sortedgenderTicket=genderTicketTranspose.sort_values([0])
x=[]
y=[]
zStatus=[]

for index, row in sortedgenderTicket.iterrows():
    x.append(row[0])
    y.append(row[1])
    zStatus.append(index)
    
plt.plot(x,zStatus)
plt.plot(y,zStatus)
 
plt.xlabel("Number of Ticket")
plt.ylabel("Ticket Status")
plt.title("Ticket Status with Gender")
plt.legend(["Female","Male"])
plt.show()
genderTicket

# probability of female and male with regard to ticket status

In [ ]:
genderTicketAggregated= before_drop.groupby(['Tiket Status ','gender'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','Nationality','Fligth_ID','cabin_class of service','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=genderTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result

genderDf = pd.DataFrame(columns=["Index","Gender","Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
genderDf[genderDf.columns[0]] = ["Female", "Male","probability"]
test=[]
test1=[]
totalFemale=0
totalMale=0

exchange=0
refunde=0
tused=0
tnotused=0
totalProb=[]
test=[]
for index, row in result.iterrows():
    test.append(row[0])
i=0;
for i in range(len(test)):
    exchange=test[i]+test[i+1]
    totalProb.append(exchange)
    refunde=test[i+2]+test[i+3]
    totalProb.append(refunde)
    tused=test[i+4]+test[i+5]
    totalProb.append(tused)
    tnotused=test[i+6]+test[i+7]
    totalProb.append(tnotused)
    break

i=0
for index, row in result.iterrows():
    if(i%2==0):
        totalFemale=row[0]+totalFemale
    else:
        totalMale=row[0]+totalMale
    i=i+1

for index, row in result.iterrows():
    if(i%2==0):
        test1.append((row[0]/totalFemale))
    else:
        test1.append((row[0]/totalMale))
    i=i+1
    
mergedData=[]
total=totalFemale+totalMale
j=0
for i in range(len(test)):
    if(j%2==0):
        mergedData.append(test[j]) 
        mergedData.append(test[j+1])
    else:
        mergedData.append((test[j] + test[j-1])/total)
    j=j+1
       

j=0
k=0
for i in range(len(mergedData)):
    this_column = genderDf.columns[i+1]
    genderDf[this_column] = [test1[k], test1[k+1],mergedData[j+2]]
    j=j+3
    k=k+2
    if(i==3):
        break

genderDf.to_excel("GenderTicketStatusAggregationResult.xlsx",index=False)
genderTicket=pd.read_excel('GenderTicketStatusAggregationResult.xlsx')
genderTicketTranspose=genderTicket.T
genderTicketTranspose=genderTicketTranspose.iloc[1: , :]
sortedgenderTicket=genderTicketTranspose.sort_values([0])
genderTicket

genderTicket

In [ ]:
# before_drop

# Class of Service with Ticket Status Aggregation

In [ ]:
classOfServiceTicketAggregated= before_drop.groupby(['Tiket Status ','cabin_class of service'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','Nationality','Fligth_ID','gender','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=classOfServiceTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
classDf = pd.DataFrame(columns=["Index","Class","Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
classDf[classDf.columns[0]] = ["Business", "Economic"]
test=[]
for index, row in result.iterrows():
    test.append(row[0])
j=0;
for i in range(len(test)):
    this_column = classDf.columns[i+1]
    classDf[this_column] = [test[j], test[j+1]]
    j=j+2
    if(i==3):
        break
classDf.to_excel("ClassOfServiceTicketStatusAggregationResult.xlsx",index=False)
classTicket=pd.read_excel('ClassOfServiceTicketStatusAggregationResult.xlsx')
classTicket
classTicketTranspose=classTicket.T
classTicketTranspose=classTicketTranspose.iloc[1: , :]
sortedclassTicket=classTicketTranspose.sort_values([0])
x=[]
y=[]
zStatus=[]

for index, row in sortedclassTicket.iterrows():
    x.append(row[0])
    y.append(row[1])
    zStatus.append(index)
    
fig = plt.figure(figsize = (10, 5))
plt.bar(zStatus,y)
plt.bar(zStatus,x) 
plt.xlabel("Number of Ticket")
plt.ylabel("Ticket Status")
plt.title("Ticket Status with Class of Service")
plt.legend(["Economic","Business"])
plt.show()
classTicket

# probability of Business class and Economy class with regard to ticket status

In [ ]:
classOfServiceTicketAggregated= before_drop.groupby(['Tiket Status ','cabin_class of service'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','Nationality','Fligth_ID','gender','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=classOfServiceTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
classDf = pd.DataFrame(columns=["Index","Class","Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
classDf[classDf.columns[0]] = ["Business", "Economic"]

secondData=[]
totalBusiness=0
totalEconomy=0

i=0
for index, row in result.iterrows():
    if(i%2==0):
        totalBusiness=row[0]+totalBusiness
    else:
        totalEconomy=row[0]+totalEconomy
    i=i+1

for index, row in result.iterrows():
    if(i%2==0):
        secondData.append((row[0]/totalBusiness))
    else:
        secondData.append((row[0]/totalEconomy))
    i=i+1
    

j=0;
for i in range(len(secondData)):
    this_column = classDf.columns[i+1]
    classDf[this_column] = [secondData[j], secondData[j+1]]
    j=j+2
    if(i==3):
        break

classDf

# Nationality with Ticket Status Aggregation

In [ ]:
NationalityTicketAggregated= before_drop.groupby(['Tiket Status ','Nationality'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','cabin_class of service','Fligth_ID','gender','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=NationalityTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
nationalityDf = pd.DataFrame(columns=["Index","Nationality", "Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
nationalityDf[nationalityDf.columns[0]] = ["Foreign", "Local"]
test=[]
for index, row in result.iterrows():
    test.append(row[0])
j=0;
for i in range(len(test)):
    this_column = nationalityDf.columns[i+1]
    nationalityDf[this_column] = [test[j], test[j+1]]
    j=j+2
    if(i==3):
        break
nationalityDf.to_excel("NationalityTicketStatusAggregationResult.xlsx",index=False)
nationalityTicket=pd.read_excel('NationalityTicketStatusAggregationResult.xlsx')
# nationalityTicket.T.sort_values(by=index,axis=1)
nationalityTranspose=nationalityTicket.T
nationalityTranspose=nationalityTranspose.iloc[1: , :]
sortedNationality=nationalityTranspose.sort_values([0])
xForeign=[]
yLocalcal=[]
zStatus=[]

for index, row in sortedNationality.iterrows():
    xForeign.append(row[0])
    yLocalcal.append(row[1])
    zStatus.append(index)
    
# plt.plot(xForeign,zStatus)
# plt.plot(yLocalcal,zStatus)
 
# plt.xlabel("Number of Ticket")
# plt.ylabel("Ticket Status")
# plt.title("Ticket Status with Nationality")
# plt.legend(["Foriegn","Local"])
# plt.show
fig = plt.figure(figsize = (10, 5))
plt.bar(zStatus,yLocalcal)
plt.bar(zStatus,xForeign) 
plt.xlabel("Ticket Status")
plt.ylabel("Number of Ticket")
plt.title("Ticket Status with Nationality")
plt.legend(["Local","Foriegn"])
plt.show()
nationalityTicket

# probability of Foreign and Local with regard to ticket status

In [ ]:
NationalityTicketAggregated= before_drop.groupby(['Tiket Status ','Nationality'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','cabin_class of service','Fligth_ID','gender','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=NationalityTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
nationalityDf = pd.DataFrame(columns=["Index","Nationality", "Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
nationalityDf[nationalityDf.columns[0]] = ["Foreign", "Local"]

secondData=[]
totalForiegn=0
totalLocal=0

i=0
for index, row in result.iterrows():
    if(i%2==0):
        totalForiegn=row[0]+totalForiegn
    else:
        totalLocal=row[0]+totalLocal
    i=i+1

for index, row in result.iterrows():
    if(i%2==0):
        secondData.append((row[0]/totalForiegn))
    else:
        secondData.append((row[0]/totalLocal))
    i=i+1
    

j=0;
for i in range(len(secondData)):
    this_column = nationalityDf.columns[i+1]
    nationalityDf[this_column] = [secondData[j], secondData[j+1]]
    j=j+2
    if(i==3):
        break
nationalityDf

# Flight Number with Ticket Status Aggregation

In [ ]:
FlightNumberTicketAggregated= before_drop.groupby(['Tiket Status ','Fligth_ID'],as_index=True).count()
tobedrop=['Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','Nationality','cabin_class of service','gender','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=FlightNumberTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
flightDf = pd.DataFrame(columns=["Index","Flight Number", "Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
flightDf[flightDf.columns[0]] = ["ET114", "ET124","ET168", "ET266","ET268"]
test=[]
for index, row in result.iterrows():
    test.append(row[0])   
j=0;
for i in range(len(test)):
    this_column = flightDf.columns[i+1]
    flightDf[this_column] = [test[j], test[j+1],test[j+2], test[j+3],test[j+4]]
    j=j+5
    if(i==3):
        break
flightDf.to_excel("FlightNumberTicketStatusAggregationResult.xlsx",index=False)
flightNumberTicket=pd.read_excel('FlightNumberTicketStatusAggregationResult.xlsx')
flightNumberTicket
flightNumberTicketTranspose=flightNumberTicket.T
flightNumberTicketTranspose=flightNumberTicketTranspose.iloc[1: , :]
sortedflightNumberTicket=flightNumberTicketTranspose.sort_values([0])
a=[]
b=[]
c=[]
d=[]
e=[]
zStatus=[]

for index, row in sortedflightNumberTicket.iterrows():
    a.append(row[0])
    b.append(row[1])
    c.append(row[2])
    d.append(row[3])
    e.append(row[4])
    zStatus.append(index)
    
plt.plot(a,zStatus)
plt.plot(b,zStatus)
plt.plot(c,zStatus)
plt.plot(d,zStatus)
plt.plot(e,zStatus)
 
plt.xlabel("Number of Ticket")
plt.ylabel("Ticket Status")
plt.title("Ticket Status with Flight Number")
plt.legend(["ET114","ET124","ET168","ET268","ET266"])
plt.show()
print(test)
flightNumberTicket


# Age Status with Ticket Status Aggregation

In [ ]:
AgeTicketAggregated= before_drop.groupby(['Tiket Status ','Age'],as_index=True).count()
tobedrop=['Fligth_ID','N_Gender','N_Nationality','N_FlightID','N_CabinClass','N_TicketStatus','N_ReservationStatus','N_ReservationDate','Nationality','cabin_class of service','gender','Reservation Status','Tiketed_Date','Date_of_Reservation']
result=AgeTicketAggregated.drop(tobedrop,axis=1)
result
# Assign row as column headers
result
ageDf = pd.DataFrame(columns=["Index","Age", "Exchanged","Refunded","Tiketed Not Used","Ticketed Used"]).set_index('Index')
ageDf[ageDf.columns[0]] = ["Adult", "Youth","Senior"]
test=[]
for index, row in result.iterrows():
    test.append(row[0])   
j=0;
for i in range(len(test)):
    this_column = ageDf.columns[i+1]
    ageDf[this_column] = [test[j], test[j+1],test[j+2]]
    j=j+3
    if(i==3):
        break
ageDf.to_excel("AgeTicketStatusAggregationResult.xlsx",index=False)
AgeTicket=pd.read_excel('AgeTicketStatusAggregationResult.xlsx')
AgeTicket
AgeTicketTranspose=AgeTicket.T
AgeTicketTranspose=AgeTicketTranspose.iloc[1: , :]
sortedAgeTicketTranspose=AgeTicketTranspose.sort_values([0])
xForeign=[]
yLocalcal=[]
zStatus=[]

for index, row in sortedAgeTicketTranspose.iterrows():
    xForeign.append(row[0])
    yLocalcal.append(row[1])
    zStatus.append(index)
    
fig = plt.figure(figsize = (10, 5))
plt.bar(zStatus,xForeign) 
plt.bar(zStatus,yLocalcal)
plt.xlabel("Ticket Status")
plt.ylabel("Number of Ticket")
plt.title("Ticket Status with Nationality")
plt.legend(["Local","Foriegn"])
plt.show()
AgeTicket

# Test prediction 

In [ ]:

def pridictData(age,gender,nationality,flightNo,cabinClass,date):
    data=[]
    data.append(age) 
    data.append(gender)
    data.append(nationality)
    data.append(flightNo)
    data.append(cabinClass)
    data.append(date)
    result=model.predict([data])
    return result
    
# 'N_Age','N_Gender','N_Nationality','N_FlightID','N_CabinClass','Tiketed_Date'    
def dataPredictor():
    if(agemenu.get()=="Adult"):
        N_Age=0
    elif(agemenu.get()=="Sineor"):
        N_Age=1
    else:
        N_Age=1
    
    if(gendermenu.get()=="Female"):
        N_Gender=0
    else:
        N_Gender=1
        
    if(nationalitymenu.get()=="Foriegn"):
        N_Nationality=0
    else:
        N_Nationality=1   
    
    if(classmenu.get()=="Business"):
        N_CabinClass=0
    else:
        N_CabinClass=1 
    months=['Quarter 1', 'Quarter 2', 'Quarter 3', 'Quarter 4']
   
    i=1
    for month in months:
        if(month==monthmenu.get()):
            N_ReservationDate=i
        i=i+1
    flightNumbers=["ET114","ET124","ET168","ET268","ET266"]
   
    j=1
    for month in flightNumbers:
        if(month==flightNomenu.get()):
            N_FlightID=j
        j=j+1
#     N_Age=int()
#     N_Gender=int(gendermenu.get())
#     N_Nationality=int(nationalitymenu.get())
#     N_FlightID=int(flightNomenu.get())
#     N_CabinClass=int(classmenu.get())
#     N_ReservationDate=int(monthmenu.get())
# 0 Exchanged
# 1 Refunded              
# 2 Tiketed  Not Used             
# 3 Tickete Used

    result=pridictData(N_Age,N_Gender,N_Nationality,N_FlightID,N_CabinClass,N_ReservationDate)
    if(result[0]==0):
        lblMessege.config(text="The result shows that, this ticket will be Exchanged")
    elif(result[0]==1):
        lblMessege.config(text="The result shows that, this ticket will be Refunded")
    elif(result[0]==2):
        lblMessege.config(text="The result shows that, this ticket will be Tiket Not Used")
    else:
        lblMessege.config(text="The result shows that, this ticket will be Ticket used")
#     check if the text entry is empty or not
        
        
        
def ErrorMessage(message):
    
    lblError.config(text=message)


window = Tk()

window.title("Welcome to Ticket used prediction system ")

window.geometry('800x500')

lbl1 = Label(window, text="Let us predict the ticket status",font=(50),fg="green")

lbl1.grid(column=1, row=1,padx=10, pady=10)


agelbl = Label(window, text="Age",font=(12),fg="green")

agelbl.grid(column=0, row=4,padx=20, pady=10)

agelbl.grid(column=0, row=4,padx=20, pady=10)

agemenu= StringVar()
agemenu.set("Adult")
drop= OptionMenu(window, agemenu, "Adult", "Youth","Sineor")
drop.grid(row=4, column=1,padx=20, pady=10)
drop.config(width = 25)
drop.config(bg = "White")
drop.grid()


genderlbl = Label(window, text="Gender",font=(12),fg="green")

genderlbl.grid(column=0, row=5,padx=20, pady=5)

genderlbl.grid(column=0, row=5,padx=20, pady=5)

gendermenu= StringVar()
gendermenu.set("Male")
drop= OptionMenu(window, gendermenu, "Male", "Female")
drop.grid(row=5, column=1,padx=20, pady=10)
drop.config(width = 25)
drop.config(bg = "White")
drop.grid()



nationalitylbl = Label(window, text="Nationality",font=(12),fg="green")

nationalitylbl.grid(column=0, row=6,padx=20, pady=2)

nationalitylbl.grid(column=0, row=6,padx=20, pady=2)

nationalitymenu= StringVar()
nationalitymenu.set("Local")
drop= OptionMenu(window, nationalitymenu, "Local", "Foriegn")
drop.grid(row=6, column=1,padx=20, pady=10)
drop.config(width = 25)
drop.config(bg = "White")
drop.grid()


N_FlightIDlbl = Label(window, text="Flight ID",font=(12),fg="green")

N_FlightIDlbl.grid(column=0, row=7,padx=20, pady=2)

N_FlightIDlbl.grid(column=0, row=7,padx=20, pady=2)

flightNomenu= StringVar()
flightNomenu.set("ET114")
drop= OptionMenu(window, flightNomenu, "ET114","ET124","ET168","ET268","ET266")
drop.grid(row=7, column=1,padx=20, pady=10)
drop.config(width = 25)
drop.config(bg = "White")
drop.grid()


N_CabinClasslbl = Label(window, text="Cabin Class",font=(12),fg="green")

N_CabinClasslbl.grid(column=0, row=8,padx=20, pady=2)

N_CabinClasslbl.grid(column=0, row=8,padx=20, pady=2)

classmenu= StringVar()
classmenu.set("Economy")
drop= OptionMenu(window, classmenu, "Economy", "Business")
drop.grid(row=8, column=1)
drop.config(width = 25)
drop.config(bg = "White")
drop.grid()


N_ReservationDatelbl = Label(window, text="Reservation Month",font=(12),fg="green")

N_ReservationDatelbl.grid(column=0, row=9,padx=20, pady=2)

N_ReservationDatelbl.grid(column=0, row=9,padx=20, pady=2)

monthmenu= StringVar()
monthmenu.set("Quarter 1")
drop= OptionMenu(window, monthmenu, 'Quarter 1', 'Quarter 2', 'Quarter 3', 'Quarter 4')
drop.grid(row=9, column=1,padx=20, pady=10)
drop.config(width = 25)
drop.config(bg = "White")
drop.grid()


btn = Button(window, text="pridict",font=(12), command=dataPredictor, bg="green", fg="white")

btn.grid(column=1, row=10,padx=0)

lblMessege=Label(window, text=" ",font=(18), fg="blue")

lblMessege.grid(column=1, row=11,padx=0, pady=0)


window.mainloop()